# Vanilla Neural Network

In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable

In [2]:
cuda = torch.cuda.is_available() # True if cuda is available, False otherwise
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor
print('Training on %s' % ('GPU' if cuda else 'CPU'))

Training on GPU


Let's load the MNIST data set.

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(mean=(.5, .5, .5), std=(.5, .5, .5))])
train_data = torchvision.datasets.MNIST(root='../data/', train=True, transform=transform, download=True)
test_data = torchvision.datasets.MNIST(root='../data/', train=False, transform=transform, download=True)

Processing...
Done!


Create a *loader* to feed the data batch by batch during training.

In [4]:
batch = 100
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch)

Now, we define the **feedforward neural network**.

In [5]:
net = torch.nn.Sequential(
        nn.Linear(28 * 28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10))

#net = net.cuda() if cuda else net
net = net.type(FloatTensor)

We define the loss function and the optimization scheme (here **Adam**) for the neural network.

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=net.parameters(), lr=0.001)

Finally, we train the classifier.

In [7]:
epochs = 5
train_size = int(train_data.train_labels.size()[0])
test_size = int(test_data.test_labels.size()[0])

accuracy = 0.

for i in range(epochs):
    # train network
    for j, (images, labels) in enumerate(train_loader):
        # map tensor from (batch, 1, 28, 28) to (batch, 28 * 28)
        #images = Variable(images.view(batch, -1)).cuda() if cuda else Variable(images.view(batch, -1))
        #labels = Variable(labels).cuda() if cuda else Variable(labels)
        #print type(labels.data)
        images = Variable(images.view(batch, -1)).type(FloatTensor)
        labels = Variable(labels).type(LongTensor)

        net.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
            
    # test network
    for images, labels in test_loader:
        #images = Variable(images.view(batch, -1)).cuda() if cuda else Variable(images.view(batch, -1))
        #labels = Variable(labels).cuda() if cuda else Variable(labels)
        images = Variable(images.view(batch, -1)).type(FloatTensor)
        labels = Variable(labels).type(LongTensor)
        outputs = net(images)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        accuracy += torch.sum(torch.eq(predicted, labels).float()).data[0] / test_size
    print('[TEST] Epoch %i/%i loss: %.2f, accuracy: %.3f' % (i + 1, epochs, loss.data[0], accuracy))
    accuracy = 0.

[TEST] Epoch 1/5 loss: 0.30, accuracy: 0.940
[TEST] Epoch 2/5 loss: 0.12, accuracy: 0.964
[TEST] Epoch 3/5 loss: 0.18, accuracy: 0.962
[TEST] Epoch 4/5 loss: 0.11, accuracy: 0.969
[TEST] Epoch 5/5 loss: 0.09, accuracy: 0.976
